This code gets Jupiter's position for a given ScW or list of ScWs, also accounting for INTEGRAL's position in orbit (parallax)

In [36]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astroquery.heasarc
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.jplhorizons import Horizons
from collections import defaultdict
from astropy.time import Time
from astropy.io import fits
import requests

In [37]:
def get_integral_position(obs_date):
    url = f"https://www.astro.unige.ch/mmoda/dispatch-data/gw/scsystem/api/v1.0/sc/{obs_date}/0/0"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        integral_ra = data['ephs']['ra'] 
        integral_dec = data['ephs']['dec']
        integral_alt = data['ephs']['alt']
        return integral_ra, integral_dec, integral_alt
    else:
        raise ValueError("Error fetching INTEGRAL data")


In [144]:
jcoords = np.loadtxt("../data/Jupiter-ScWs.txt", delimiter=",", skiprows=1, usecols=(4, 5))
jupiter_ra = jcoords[:, 0]
jupiter_dec = jcoords[:, 1]
jupiter_coords = SkyCoord(ra=jupiter_ra, dec=jupiter_dec, unit="deg")

jdates = np.loadtxt("../data/Jupiter-ScWs.txt", delimiter=",", skiprows=1, usecols=(2,), dtype=str)
jdates = [Time(jd, format="mjd").datetime for jd in jdates]

In [148]:
fits_files = np.sort(os.listdir('../data/JupiterIMG_FITS_15_30/')) # list of the jupiter SCW FITS files
scws = [f[:16] for f in fits_files] # list of the SCW names

scw_dates = []
scw_ra = []
scw_dec = []
scw_jra = []
scw_jdec = []
expected_jra = []
expected_jdec = []

parallax = True

for f in fits_files[:1]:
    with fits.open('../data/JupiterIMG_FITS_15_30/' + f) as hdu:
        header = hdu[2].header
        obs_date = header['DATE-OBS']
        end_date = header['DATE-END']
        ra = header['CRVAL1']
        dec = header['CRVAL2']

        scw_dates.append(obs_date)
        scw_ra.append(ra)
        scw_dec.append(dec)

        date_obs = Time(obs_date).datetime

        # Find closest Jupiter position
        pointing_coords = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)

        closest_idx = jupiter_coords.separation(pointing_coords).argmin()
        closest_idx = np.argmin(np.abs([jdates[i] - date_obs for i in range(len(jdates))]))
        
        j_ra = jupiter_ra[closest_idx]
        j_dec = jupiter_dec[closest_idx]

        expected_jra.append(j_ra)
        expected_jdec.append(j_dec)

        epochs = Time(obs_date).jd
        jupiter = Horizons(id='599', location='@0', epochs=epochs)
        eph = jupiter.ephemerides()
        jra = eph['RA']
        jdec = eph['DEC']
        jdist = eph['delta'] * u.au.to(u.km) * u.km

        if parallax:

            integral_ra, integral_dec, integral_alt = get_integral_position(obs_date)

            integral_position = SkyCoord(ra=integral_ra*u.deg, dec=integral_dec*u.deg, distance=integral_alt*u.km).transform_to('fk5')
            jupiter_position = SkyCoord(ra=jra, dec=jdec, distance=jdist).transform_to('fk5')

            jupiter_relative_position = jupiter_position.cartesian - integral_position.cartesian

            relative_position = SkyCoord(x=jupiter_relative_position.x, 
                                y=jupiter_relative_position.y, 
                                z=jupiter_relative_position.z, 
                                representation_type='cartesian').transform_to(integral_position.frame)

            jra = relative_position.ra.value
            jdec = relative_position.dec.value

        scw_jra.append(jra)
        scw_jdec.append(jdec)

In [159]:
print('Jupiter RA:', jra)
print('Jupiter DEC:', jdec)

print('Expected Jupiter RA:', expected_jra)
print('Expected Jupiter DEC:', expected_jdec)

print('SCW RA:', ra)
print('SCW DEC:', dec)

Jupiter RA: [133.5407836]
Jupiter DEC: [18.12350495]
Expected Jupiter RA: [196.39812]
Expected Jupiter DEC: [-5.57746]
SCW RA: 191.178115844727
SCW DEC: 0.152548849582672


Save to file

In [138]:
with open('../data/scw_jup_positions.txt', 'w') as f:
    f.write("SCW,Date,RA,Dec,Jupiter_RA,Jupiter_Dec\n")
    for scw, date, ra, dec, jra, jdec in zip(scws, scw_dates, scw_ra, scw_dec, scw_jra, scw_jdec):
        f.write(f"{scw},{date},{ra},{dec},{jra},{jdec}\n")